## Analysis Notebook

### Import libraries

In [4]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

### Read Data

In [7]:
df_playlist = pd.read_csv('data/playlist_data.csv')
df_playlist

,#,Song,Artist,BPM,Camelot,Energy,Added At,Duration,Popularity,Genres,...,Acoustic,Instrumental,Valence,Speech,Live,Loud (Db),Key,Time Signature,Spotify Track Id,ISRC
0,1,Monster vs. Angel,"WDL,mawe",140,2A,56,2026-01-18,03:25,53,NaN,...,86,54,31,0,10,-6,D#/E♭ Minor,4,0stKYWa4tQ0jFXmOUDFbxM,SEWEE1501902
1,2,Tread On Me,Matt Maeson,140,8B,49,2026-01-18,03:30,47,NaN,...,44,0,54,0,20,-8,C Major,4,750amxX4DpEsA5N2IcAtm5,USAT21900732
2,3,Glue Myself Shut,Noah Kahan,73,3B,31,2026-01-18,03:16,71,NaN,...,88,0,70,0,30,-8,C♯/D♭ Major,4,67q8yivDoOPXCYodi1zTix,USUM72009075
3,4,Alright,Demons Of Ruby Mae,180,8A,87,2026-01-18,02:46,3,NaN,...,0,0,48,0,0,-5,A Minor,4,18jFJ06fHCUdejRzuI89Tp,UKX3E2100001
4,5,Loud(y),Lewis Del Mar,140,6A,56,2026-01-18,04:09,32,NaN,...,26,0,50,0,10,-7,G Minor,4,2Df8M8sO1zhC71p2XQRwpj,USSM11509018
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70,71,Snap Out Of It,Arctic Monkeys,130,4A,64,2026-01-22,03:13,80,"indie, garage rock",...,25,0,87,0,10,-6,F Minor,4,0NdTUS4UiNYCNn5FgVqKQY,GBCEL1300371
71,72,Wires,The Neighbourhood,146,6A,59,2026-01-23,03:12,45,NaN,...,0,33,28,0,30,-6,G Minor,4,4ObATYHHvXXlJJQ4rPKU2s,USSM11204648
72,73,When You Were Young,The Killers,130,1B,99,2026-01-23,03:40,79,alternative rock,...,0,5,32,10,20,-3,B Major,4,70wYA8oYHoMzhRRkARoMhU,USUM70605164
73,74,Finding Me,Vertical Horizon,99,2B,93,2026-01-23,04:32,33,NaN,...,0,0,44,0,20,-5,F#/G♭ Major,4,1kFokK4qD3mAXfM0BUHkXS,USRC19900052
